#  Tutorial: CNLS_VRS model

- Author   : Sheng Dai (sheng.dai@aalto.fi)
- Objective: Estimates a variable returns to scale (VRS) production function model and overall economic efficiency
- Data     : data genrate pprocess (DGP) 
- Date     : April 16, 2020

In [1]:
import CNLS_VRS
import numpy as np
import pandas as pd
from cvxopt import matrix

In [2]:
## Generate data
## Number of DMUs
m = 50     

## set seed
np.random.seed(1)

## x1~U[1,10]
x1 = np.random.uniform(1, 10, m)

## composite error term
#  v ~  N(0, 0.708^2)
#  u ~ |N(0, 1.174^2)|
y = 3 + x1**0.5+np.log(x1)+np.random.normal(0,0.708,m)- abs(np.random.normal(0,1.174,m))

x = np.asmatrix(x1).T

In [3]:
## calculate the VRS model; 
##           call function cnls_vrs()
res_cnls = CNLS_VRS.cnls_vrs(x, y, 'MOSEK')

## results: yhat and beta
yhat  = res_cnls[:m]
beta  = res_cnls[m:2*m]

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : QO (quadratic optimization problem)
  Constraints            : 2550            
  Cones                  : 0               
  Scalar variables       : 100             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Quadratic to conic reformulation started.
Quadratic to conic reformulation terminated. Time: 0.00    
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.03    
Problem
  Name                   :        

In [4]:
## calculate the residuals (eps)
eps   = matrix(0.0, (m,1))
for i in range(m):
    eps[i]   = y[i] - yhat[i]  

In [5]:
## calculate the constant term (alpha)
alpha = matrix(0.0, (m,1))
for i in range(m):
 alpha[i] = y[i] - eps[i] - beta[i] * x1[i]

In [6]:
alpha = pd.DataFrame(alpha)
beta  = pd.DataFrame(beta)
eps   = pd.DataFrame(eps)

results = pd.concat([alpha, beta, eps], axis=1) 

var_name = ['Alpha', 'Beta', 'Residual']

results.columns = var_name

In [7]:
## print estimates
print(results)

       Alpha      Beta  Residual
0   3.208897  0.583749  0.049887
1   7.116232  0.028273 -0.659485
2   0.547997  2.607770  0.135045
3   3.208887  0.583752  0.713273
4   3.208873  0.583756 -0.776738
5   3.208868  0.583759  0.402999
6   3.208876  0.583755  1.881322
7   3.208892  0.583750  0.031980
8   3.208894  0.583750 -0.762829
9   3.208903  0.583748 -0.284391
10  3.208898  0.583749 -0.216973
11  7.116193  0.028278  0.776131
12  3.208879  0.583754  0.401088
13  7.116256  0.028270  0.260459
14  1.425137  1.904080  0.269726
15  4.878045  0.346459  1.723684
16  3.208897  0.583749  0.112253
17  3.208904  0.583748  0.141324
18  3.208872  0.583757  0.478789
19  3.208878  0.583754 -0.828675
20  7.116237  0.028272 -0.661174
21  7.388173  0.000007  0.057953
22  3.208889  0.583751 -0.577463
23  7.116218  0.028275  0.563203
24  7.116253  0.028270  0.522920
25  7.116261  0.028269  0.378926
26  3.208868  0.583759  0.547280
27  2.826825  0.866441  1.054324
28  3.208874  0.583756 -1.313305
29  7.1162